In [ ]:
# Imports
import numpy as np
import seaborn as sns
from skmultiflow.drift_detection.adwin import ADWIN
from matplotlib import pyplot as plt
import pandas as pd
from tqdm import tqdm 
import imageio.v2 as imageio
from mpl_toolkits.axes_grid1 import ImageGrid
import matplotlib.cm as cm
import matplotlib.animation as animation
import cv2
from time import sleep
import gc
from glob import glob
import joblib
from joblib import Parallel, delayed
import warnings 
warnings.filterwarnings("ignore")
import os
from random import sample, seed, randint, random, shuffle, choice, choices, uniform, gauss, triangular
from PIL import Image, ImageOps 
from pandas_datareader import data as pdr
import pandas as pd
import shutil
from collections import OrderedDict
os.chdir("/home/smoothjazzuser/videogame-anomoly/MNAD/")
# get current dir
os.getcwd()
import numpy as np
import os
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torch.nn.init as init
import torch.utils.data as data
import torch.utils.data.dataset as dataset
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.autograd import Variable
import torchvision.utils as v_utils
import matplotlib.pyplot as plt
import cv2
import math
from collections import OrderedDict
import copy
import time
from model.utils import DataLoader
from model.final_future_prediction_with_memory_spatial_sumonly_weight_ranking_top1 import *
from model.Reconstruction import *
from sklearn.metrics import roc_auc_score
from utils import *
import random
import subprocess
import argparse
import matplotlib.pyplot as plt
from time import sleep
import matplotlib.image as mpimg
import shutil
import cv2
from scipy.stats import percentileofscore
import PIL
from joblib import Parallel, delayed
import shutil

In [ ]:
def normalize_list(list:list):
        min_val = min(list)
        max_val = max(list)
        return [(x-min_val)/(max_val-min_val) for x in list]
def load_(filename):
        i = plt.imread(filename)
        return  i
def load_frames(i, h, w):
    return np.uint8(cv2.resize(imageio.imread(f"/tmp/{i}.jpg"), (h,w)))
def normalize_array(array:np.ndarray):
    min_val = np.min(array)
    max_val = np.max(array)
    return (array-min_val)/(max_val-min_val)
def split_list(list:list, n:int):
    return [list[i:i+n] for i in range(0, len(list), n)]
def plot_images(i, x, diffs, ground_truth, label_plot, preds, points):
        window = 50
        if i > 50: window = i - 50
        else: window = 0
        
        #plot preds
        fig = plt.figure()
        fig.set_size_inches(10, 10, forward=True)
        ax = fig.add_subplot(2,2,1)
        ax.set_title(f"prediction {window}:{i}")
        ax.imshow(preds, cmap='gray') 

        #plot diffs
        ax = fig.add_subplot(2,2,2)
        ax.set_title(f"diff {window}:{i}")
        ax.imshow(diffs, cmap='gray')

        #plot ground_truth
        ax = fig.add_subplot(2,2,4)
        ax.set_title(f"ground truth {window}:{i}")
        ax.imshow(ground_truth, cmap='gray')

        #plot anomaly score
        ax = fig.add_subplot(2,2,3)
        ax.set_title(f"anomaly score {window}:{i}")
        ax.plot(np.arange(len(x[window:i])), x[window:i], color='black', label='anomaly score', linewidth=0.6)
        ax.scatter(np.arange(len(x[window:i])), x[window:i], color='black', label='anomaly score', s=4)
        ax.scatter(np.arange(len(label_plot['x'][window:i])), label_plot['y'][window:i], color='red', marker='x', s=4)

        # find all search.points that are in the window
        matching = [(i,points[x]) for i, x in enumerate(np.arange(window, i)) if x in points.index]
        # plot these points with y-values = data y values
        ax.scatter([x[0] for x in matching], [x[1] for x in matching], label='Flagged points', color='purple', s=14, alpha=0.5, marker='o')


        ax.legend([metric])
        fig.tight_layout()
        fig.subplots_adjust(wspace=0, hspace=0)
        #set ax to show 50 x values
        ax.set_xlim(0, 50)
        ax.set_ylim(0, 1)
        fig.savefig(f"/tmp/{i}.jpg", bbox_inches='tight', pad_inches=0)
        
        fig.clear()
        plt.close(fig)
        del ax
        del window
        del fig
        return None
def percentile  (array, percentile):
    percentile = np.squeeze(percentile)
    array = np.squeeze(array)
    return np.percentile(array, percentile)
def copy_frames(folder, destination, keep_every):
            files = sorted(glob(os.path.join(folder, '*.jpg')), key=lambda x: int(x.split('/')[-1].split('.')[0]))
            dest = os.path.join(destination, folder.split('/')[-1])
            for i, file in enumerate(files):
                if i % keep_every == 0:
                    shutil.copy(file, os.path.join(dest, os.path.basename(file)))
            # rename all files in the folder to be sequential, starting at 0
            files = sorted(glob(os.path.join(dest, '*.jpg')), key=lambda x: int(x.split('/')[-1].split('.')[0]))
            for i, file in enumerate(files):
                os.rename(file, os.path.join(dest, str(i).zfill(3)+'.jpg'))

            # remove frames until len(files) is a multiple of 5
            files = sorted(glob(os.path.join(dest, '*.jpg')), key=lambda x: int(x.split('/')[-1].split('.')[0]))
            while len(files) % 1 != 0:
                os.remove(files[-1])
                files = sorted(glob(os.path.join(dest, '*.jpg')), key=lambda x: int(x.split('/')[-1].split('.')[0]))
            del files, dest, folder, destination, keep_every
mse = nn.MSELoss(reduction='none')
def bl (x, y, ix, id):
    x = cv2.GaussianBlur(x, (5,5), 0)
    y = cv2.GaussianBlur(y, (5,5), 0)

    x = np.squeeze(x)
    y = np.squeeze(y)
    
    x = np.fft.fft2(x)
    y = np.fft.fft2(y)
    x = np.fft.fftshift(x)
    y = np.fft.fftshift(y)
    #set all values below 0.1 to 0
    x_h = np.percentile(np.abs(x), 100)
    y_h = np.percentile(np.abs(y), 100)
    x_l = np.percentile(np.abs(x), 90)
    y_l = np.percentile(np.abs(y), 90)
    x = np.where((np.abs(x) > x_h) &  (np.abs(x) < x_l), 0, x)
    y = np.where((np.abs(y) > y_h) &  (np.abs(y) < y_l), 0, y)
    x = np.fft.ifftshift(x)
    y = np.fft.ifftshift(y)
    x = np.fft.ifft2(x)
    y = np.fft.ifft2(y)
    
    x = np.abs(x)
    y = np.abs(y)

    x = cv2.GaussianBlur(x, (5,5), 0)
    y = cv2.GaussianBlur(y, (5,5), 0)

    d = np.abs(x-y)
    for _ in range(10):
        d = cv2.GaussianBlur(d, (11,11), 0)

    histories[str(id)].append(d)
    if len(histories[str(id)]) > 25: 
        histories[str(id)].pop(0)
    d = np.abs(d - np.mean(np.array(histories[str(id)]) , axis=0)) + 1e-9
    # for _ in range(10):
    #     d = cv2.GaussianBlur(d, (11,11), 0)
    thresh_d = np.percentile(d, 90)
    histories['t'+str(id)[1]].append(thresh_d)
    if len(histories['t'+str(id)[1]]) > 10:
        histories['t'+str(id)[1]].pop(0)
    d = np.where(d < np.mean(histories['t'+str(id)[1]]), 0, d)


    # #opencv imfill
    # d = cv2.dilate(d, np.ones((3,3), np.uint8), iterations=10)
    # d = cv2.erode(d, np.ones((3,3), np.uint8), iterations=10)
    # thresh_d1 = np.percentile(d, 90)
    # thresh_d2 = np.percentile(d, 97)
    # d = np.where(d < thresh_d, 0, d)
    return d

loss_sections = 1
std_loss_correction = {i:[] for i in range(loss_sections)} #False
histories = {}
for i in range(3):
    histories[f"d{i}"] = []
    histories[f"x{i}"] = []
    histories[f"y{i}"] = []
    histories[f"t{i}"] = []

def loss_func_mse(x, y, fft=True, colors=True, method='blur', std_details=True, diff_ = True):
    global loss_sections, std_loss_correction, histories
    #split l into len(loss_sections) sections
    losses = {'x':[], 'y':[]}

    if method=='blur':
        x = x.cpu().detach().numpy()
        y = y.cpu().detach().numpy()
        # convert to double precision
        x = x.astype(np.float64)
        y = y.astype(np.float64)

        #x = np.mean(x, axis=0)
        #x = np.expand_dims( x, axis=0)
        d = np.zeros(x.shape)
        for i in range(x.shape[0]):
            d[i,:,:] = bl(x[i,:,:], y[i,:,:], f'x{i}', f'd{i}')

        plt.imsave(f"/home/smoothjazzuser/Desktop/ram/fft_x_{len(glob('/home/smoothjazzuser/Desktop/ram/*'))+1}.png", normalize_array(d.transpose(1,2,0)))
        #y = normalize_array(y)
        #x = normalize_array(x)
        #d = normalize_array(d)

        loss = np.mean(d)
        loss = torch.tensor(loss).cuda()

    return loss
class anomaly():
    def __init__(self, method, metric, std_correction, dataset):
        self.method, self.metric, self.std_correction, self.dataset = method, metric, std_correction, dataset
    ##############################################################################
    def swap_test_folders(self, inference='01'):
        os.chdir("/home/smoothjazzuser/videogame-anomoly/")
        self.inference = inference
        #check if frames exists:
        if os.path.exists('/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/testing/frames'):
            !rm -r '/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/testing/frames'

        if inference == 'all':
            !ln -s '/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/symlink_loc/' '/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/testing/frames'
        else:
            if not os.path.exists('/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/testing/frames'):
                !mkdir '/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/testing/frames/'
            !ln -s '/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/symlink_loc/'$inference '/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/testing/frames/'$inference
        ##############################################################################
        frame_labels_bugs = np.load('/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/symlink_loc/GTall/frame_labels_bugs.npy', allow_pickle=True)
        # expand dims and save again
        #frame_labels_bugs = np.expand_dims(frame_labels_bugs, axis=0)
        frame_labels_bugs[0]
        #np.save('/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/testing/frames/frame_labels_bugs.npy', frame_labels_bugs)
        if True:
            frame_labels_bugs = np.load('/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/symlink_loc/GTall/frame_labels_bugs.npy', allow_pickle=True)

            # list directories in /home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/testing/frames/ using glob
            # glob returns a list of all files in the directory
            dd = glob('/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/testing/frames/*')
            dd.sort()
            directories = [x.split('/')[-1] for x in dd if x.split('/')[-1].isdigit()]
            directories.sort()
            directories = {d:len(glob('/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/testing/frames/' + d + '/*.jpg')) for d in directories if os.path.exists('/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/testing/frames/' + d)}
            directories = OrderedDict(sorted(directories.items(), key=lambda x: int(x[0])))
            print(list(directories))
            iterate = 0
            for folder, num_frames in directories.items():
                if not os.path.exists('/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/testing/frames/' + folder):
                    !mkdir '/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/testing/frames/'$folder

                    print(folder, num_frames)
                if not os.path.exists('/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/symlink_loc/GT' + folder + '/frame_labels_bugs.npy'):
                    !mkdir '/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/symlink_loc/GT'$folder
                    # seperate the labels for each folder from the frame_labels_bugs
                    print(folder, num_frames)
                    labels = frame_labels_bugs[0][iterate:iterate+int(num_frames)]
                    iterate += num_frames
                    np.save('/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/symlink_loc/GT' + folder + '/frame_labels_bugs.npy', np.array(np.expand_dims(np.array(labels), axis=0)))
        if os.path.exists('/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/frame_labels_bugs.npy'):
            !rm '/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/frame_labels_bugs.npy'
        if inference == 'all':
            !ln -s '/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/symlink_loc/GTall/frame_labels_bugs.npy' '/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/frame_labels_bugs.npy'
        else:
            !ln -s '/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/symlink_loc/GT'$inference'/frame_labels_bugs.npy' '/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/frame_labels_bugs.npy'

        # #rename all folders with numeric names within '/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/symlink_loc/' from 01 to 100...
        if False: 
            for folder_path in glob('/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/symlink_loc/*'):
                if folder_path.split('/')[-1].isdigit():
                    new_folder_path = folder_path.split('/')
                    new_folder_path[-1] = str(int(new_folder_path[-1]) + 1).zfill(2)
                    new_folder_path = '/'.join(new_folder_path)
                    !mv $folder_path $new_folder_path

        if False: 
            for folder_path in glob('/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/training/frames/*'):
                if folder_path.split('/')[-1].isdigit():
                    new_folder_path = folder_path.split('/')
                    new_folder_path[-1] = str(int(new_folder_path[-1]) + 1).zfill(2)
                    new_folder_path = '/'.join(new_folder_path)
                    !mv $folder_path $new_folder_path

        # for len(dir_files) in /home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/training/frames/, ensure num is dividable by 5. if not, remove files until it is
        if False:######################################################3
            for folder_path in sorted(glob('/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/training/frames/*'), key=lambda x: int(x.split('/')[-1])):
                if folder_path.split('/')[-1].isdigit():
                    dir_files = sorted(glob(folder_path + '/*'), key=lambda x: int(x.split('/')[-1].split('.')[0]))
                    if len(dir_files) % 5 != 0:
                        
                        # how many files do we need to remove?
                        num_to_remove = len(dir_files) % 5
                        #print(folder_path, len(dir_files), num_to_remove, dir_files[-1])

                        # remove the last num_to_remove files
                        for i in range(num_to_remove):
                            print (dir_files[-1])
                            name = dir_files[-1-i]
                            !rm -r $name

        test = np.load('/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/frame_labels_bugs.npy', allow_pickle=True)
        print(test.shape)
        (sum(directories.values()))
    def Evaluate(self, c=3, loss_sections=1, method='blur', force_keys=False, method_ = 'recon', dataset_type='bugs'):
        os.chdir("/home/smoothjazzuser/videogame-anomoly/MNAD/")
        std_loss_correction = {i:[] for i in range(loss_sections)} #False
        prev = 0.0000001

        args = {'gpus':'','batch_size':5,'test_batch_size':1,'h':256,'w':256,'c':c,'method':method_,'t_length':1 if method_=='recon' else 5,'fdim':512,'mdim':512, 'msize':10,'alpha':0.6,'th':0.0001,'num_workers':8,'num_workers_test':1,'dataset_type':'bugs','dataset_path':'./dataset','model_dir':f'./exp/{dataset_type}/{method_}/log/model.pth','m_items_dir':f'./exp/{dataset_type}/{method_}/log/keys.pt'}
        

        os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
        gpus = "0"
        os.environ["CUDA_VISIBLE_DEVICES"]= gpus
        # gpus = ""
        # for i in range(len(args['gpus'])):
        #     gpus = gpus + args['gpus'][i] + ","
        # os.environ["CUDA_VISIBLE_DEVICES"]= gpus[:-1]

        torch.backends.cudnn.enabled = True # make sure to use cudnn for computational performance

        test_folder = args['dataset_path']+"/"+args['dataset_type']+"/testing/frames"

        # Loading dataset
        test_dataset = DataLoader(test_folder, transforms.Compose([transforms.ToTensor(),]), resize_height=args['h'], resize_width=args['w'], time_step=args['t_length']-1)

        test_size = len(test_dataset)
        print("The number of test data is %d" % test_size)

        test_batch = data.DataLoader(test_dataset, batch_size = args['test_batch_size'], shuffle=False, num_workers=args['num_workers_test'], drop_last=False)

        ###############################
        histories = {}
        for i in range(args['c']):
            histories[f"d{i}"] = []
            histories[f"x{i}"] = []
            histories[f"y{i}"] = []
            histories[f"t{i}"] = []

        try:
            #delete r all files in '/home/smoothjazzuser/Desktop/ram/'
            shutil.rmtree('/home/smoothjazzuser/Desktop/ram/', ignore_errors=True)
        except:
            pass
        
        # if method == 'none':
        #     loss_func_mse = nn.MSELoss(reduction='none')
        # elif method == 'blur':
            
        #     loss_func_mse = loss_func_mse

            
            
        #######################################################
        # Loading the trained model
        model = torch.load(args['model_dir'])
        model.cuda()
        m_items = torch.load(args['m_items_dir'])
        labels = np.load('./data/frame_labels_'+args['dataset_type']+'.npy')

        videos = OrderedDict()

        videos_list = sorted(glob(os.path.join(test_folder, '*')), key=lambda x: int(x.split('/')[-1].split('.')[0]))
        for video in videos_list:
            video_name = video.split('/')[-1]
            videos[video_name] = {}
            videos[video_name]['path'] = video
            videos[video_name]['frame'] = sorted(glob(os.path.join(video, '*.jpg')), key=lambda x: int(x.split('/')[-1].split('.')[0]))
            #videos[video_name]['frame'].sort()
            videos[video_name]['length'] = len(videos[video_name]['frame'])

        labels_list = []
        label_length = 0
        psnr_list = {}
        feature_distance_list = {}

        print('Evaluation of', args['dataset_type'])

        # Setting for video anomaly detection
        for video in sorted(videos_list, key=lambda x: int(x.split('/')[-1].split('.')[0])):
        #for video in videos_list:
            video_name = video.split('/')[-1]
            if args['method'] == 'pred':
                labels_list = np.append(labels_list, labels[0][4+label_length:videos[video_name]['length']+label_length])
            else:
                labels_list = np.append(labels_list, labels[0][label_length:videos[video_name]['length']+label_length])
            label_length += videos[video_name]['length']
            psnr_list[video_name] = []
            feature_distance_list[video_name] = []

        label_length = 0
        video_num = 0
        label_length += videos[videos_list[video_num].split('/')[-1]]['length']
        m_items_test = m_items.clone()

        model.eval()
        kkk=0
        ccc = int(test_size/args['test_batch_size'])
        diffs = []
        preds = []
        ground_truths = []
        label_list = np.load(f"/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/frame_labels_{args['dataset_type']}.npy", allow_pickle=True).tolist()[0]
        loss_hist = {i:[] for i in range(loss_sections)}
        for k,(imgs) in enumerate(test_batch):
            
            if args['method'] == 'pred':
                if k == label_length-4*(video_num+1):
                    video_num += 1
                    label_length += videos[videos_list[video_num].split('/')[-1]]['length']
            else:
                if k == label_length:
                    video_num += 1
                    label_length += videos[videos_list[video_num].split('/')[-1]]['length']

            imgs = Variable(imgs).cuda()
            
            if args['method'] == 'pred':

                if force_keys:
                    outputs, feas, updated_feas, m_items_test, softmax_score_query, softmax_score_memory, _, _, _, compactness_loss = model.forward(imgs[:,0:3*4], m_items.clone(), False)#m_items_test
                else:
                    outputs, feas, updated_feas, m_items_test, softmax_score_query, softmax_score_memory, _, _, _, compactness_loss = model.forward(imgs[:,0:3*4], m_items_test, False)#
                mse_imgs = torch.mean(loss_func_mse((outputs[0]+1)/2, (imgs[0,3*4:]+1)/2)).item()
                mse_feas = compactness_loss.item()

                # Calculating the threshold for updating at the test time
                point_sc = point_score(outputs, imgs[:,3*4:])

                # visualize the difference between reconstructed and predicted frames in image format
                #diff = np.abs((outputs[0].detach().cpu().numpy()+1)/2 - (imgs[0].detach().cpu().numpy()+1)/2).transpose(1,2,0)
                diff = np.abs((outputs[0].detach().cpu().numpy()+1)/2 - (imgs[0,3*4:].detach().cpu().numpy()+1)/2).transpose(1,2,0)
                #diff = normalize_array(diff)
                diffs.append(diff)
                pred = ((outputs[0].detach().cpu().numpy()+1)/2).transpose(1,2,0)
                #pred = normalize_array(pred)
                preds.append(pred)

            
            else:
                if force_keys:
                    outputs, feas, updated_feas, m_items_test, softmax_score_query, softmax_score_memory, compactness_loss = model.forward(imgs, m_items.clone(), False)#

                else:
                    outputs, feas, updated_feas, m_items_test, softmax_score_query, softmax_score_memory, compactness_loss = model.forward(imgs, m_items_test, False)
                mse_imgs = torch.mean(loss_func_mse((outputs[0]+1)/2, (imgs[0]+1)/2)).item()
                mse_feas = compactness_loss.item()

                # Calculating the threshold for updating at the test time
                point_sc = point_score(outputs, imgs)

                # visualize the difference between reconstructed and predicted frames in image format
                diff = np.abs((outputs[0].detach().cpu().numpy()+1)/2 - (imgs[0].detach().cpu().numpy()+1)/2).transpose(1,2,0)
                #diff = normalize_array(diff)
                diffs.append(diff)

                pred = (outputs[0].detach().cpu().numpy()+1)/2
                pred = pred.transpose(1,2,0)
                preds.append(pred)


            if  point_sc < args['th']: # or k < 5: 
                print(f"updating memory -GT {label_list[kkk]} -th {args['th']} -value {point_sc}" )
                query = F.normalize(feas, dim=1)
                query = query.permute(0,2,3,1) # b X h X w X d
                m_items_test = model.memory.update(query, m_items_test, False)
            prev = point_sc

            kkk+=1
        

            psnr_list[videos_list[video_num].split('/')[-1]].append(psnr(mse_imgs))
            feature_distance_list[videos_list[video_num].split('/')[-1]].append(mse_feas)
            print(f"pairs pred: {round(100*kkk/ccc,3)}%, mse_feas:{mse_feas}", end = "\r")


        # Measuring the abnormality score and the AUC
        anomaly_score_total_list = []
        for video in sorted(videos_list):
            video_name = video.split('/')[-1]
            anomaly_score_total_list += score_sum(anomaly_score_list(psnr_list[video_name]), 
                                            anomaly_score_list_inv(feature_distance_list[video_name]), args['alpha'])

        anomaly_score_total_list = np.asarray(anomaly_score_total_list)

        accuracy = AUC(anomaly_score_total_list, np.expand_dims(1-labels_list, 0))

        np.save(f"./exp/{args['dataset_type']}/{args['method']}/log/anomaly_score_total_list.npy", anomaly_score_total_list)
        np.save(f"./exp/{args['dataset_type']}/{args['method']}/log/anomaly_score_total_list.npy", anomaly_score_total_list)
        np.save(f"./exp/{args['dataset_type']}/{args['method']}/log/psnr_list.npy", psnr_list)
        np.save(f"./exp/{args['dataset_type']}/{args['method']}/log/feature_distance_list.npy", feature_distance_list)
        np.save(f"./exp/{args['dataset_type']}/{args['method']}/log/auc.npy", accuracy)
        np.save(f"./exp/{args['dataset_type']}/{args['method']}/log/labels_list.npy", labels_list)

        if os.path.exists(f"./exp/{args['dataset_type']}/{args['method']}/log/diffs/"):
            l = f"./exp/{args['dataset_type']}/{args['method']}/log/diffs/"
            shutil.rmtree(l)
        if os.path.exists(f"./exp/{args['dataset_type']}/{args['method']}/log/preds/"):
            l = f"./exp/{args['dataset_type']}/{args['method']}/log/preds/"
            shutil.rmtree(l)
        if os.path.exists(f"./exp/{args['dataset_type']}/{args['method']}/log/ground_truths/"):
            l = f"./exp/{args['dataset_type']}/{args['method']}/log/ground_truths/"
            shutil.rmtree(l)

        if not os.path.exists(f"./exp/{args['dataset_type']}/{args['method']}/log/diffs/"):
            os.makedirs(f"./exp/{args['dataset_type']}/{args['method']}/log/diffs/")
        if not os.path.exists(f"./exp/{args['dataset_type']}/{args['method']}/log/preds/"):
            os.makedirs(f"./exp/{args['dataset_type']}/{args['method']}/log/preds/")

        for (i,img) in enumerate(preds): 
            #use pillow to save the image in grayscale
            if args['c'] == 1:
                img = img*255
                img = img.astype(np.uint8)
                img = np.squeeze(img)
                img = PIL.Image.fromarray(img, 'L')
                img.save(f"./exp/{args['dataset_type']}/{args['method']}/log/preds/{i}.jpg")
            else:
                img = img*255
                img = img.astype(np.uint8)
                img = PIL.Image.fromarray(img, 'RGB')
                img.save(f"./exp/{args['dataset_type']}/{args['method']}/log/preds/{i}.jpg")

        for (i,img) in enumerate(diffs): 
            if args['c'] == 1:
                img = img*255
                img = img.astype(np.uint8)
                img = np.squeeze(img)
                img = PIL.Image.fromarray(img, 'L')
                img.save(f"./exp/{args['dataset_type']}/{args['method']}/log/diffs/{i}.jpg")
            else:
                img = img*255
                img = img.astype(np.uint8)
                img = PIL.Image.fromarray(img, 'RGB')
                img.save(f"./exp/{args['dataset_type']}/{args['method']}/log/diffs/{i}.jpg")


        print('The result of ', args['dataset_type'])
        print('AUC: ', accuracy*100, '%')

    def rm_train_music_copy(self):
        if not os.path.exists('/tmp/empty'):
            os.makedirs('/tmp/empty')
        !rsync -a --delete /tmp/empty/   /home/smoothjazzuser/VQ-VAE-Search-main/mel_specs_music/train/cl/
        !rsync -a --delete /tmp/empty/   /home/smoothjazzuser/VQ-VAE-Search-main/mel_specs_music/test/cl/
    def cut_dataset(self, keep_every=10, ram=True, source_path='/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/training/full', destination = '/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/training/frames', ):
        os.chdir("/home/smoothjazzuser/videogame-anomoly/")
        self.ls_destination = destination
        self.source_path = source_path
        self.destination = destination
        self.keep_every = keep_every
        self.ram = ram
        if not os.path.exists('/tmp/empty'):
            os.makedirs('/tmp/empty')
        if self.ram:
            if not os.path.exists('/home/smoothjazzuser/Desktop/ram/frames/'):
                os.makedirs('/home/smoothjazzuser/Desktop/ram/frames/')
            self.destination = '/home/smoothjazzuser/Desktop/ram/frames'
            !rsync -a --delete /tmp/empty/   /home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/training/frames/
            !rsync -a --delete /tmp/empty/   /home/smoothjazzuser/Desktop/ram/frames/
            !rm -rf /home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/training/frames
        else: 
            !rsync -a --delete /tmp/empty/   /home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/training/frames/
            #del /frames/ folder
            !rm -rf /home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/training/frames
            #create new /frames/ folder
            !mkdir /home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/training/frames
            print('Saving to RAM')
        !mkdir -p /tmp/empty

        self.folders_list = sorted(glob(os.path.join(self.source_path, '*')))
        print(len(self.folders_list))

        # create all desitnation folders, if they don't exist
        for folder in self.folders_list:
            if not os.path.exists(os.path.join(self.destination, folder.split('/')[-1])):
                os.makedirs(os.path.join(self.destination, folder.split('/')[-1]))

        

        c = Parallel(n_jobs=12)(delayed(copy_frames)(folder, self.destination, self.keep_every) for folder in self.folders_list)
        del c
        if ram:
            !ln -s /home/smoothjazzuser/Desktop/ram/frames/ /home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/training/


    def analysis(self):
        # change working directory in jupyter notebook
        os.chdir("/home/smoothjazzuser/videogame-anomoly/")
        self.anomaly_score_total_list = np.load(f"/home/smoothjazzuser/Desktop/videogame-anomoly/MNAD/exp/{self.dataset}/{self.method}/log/anomaly_score_total_list.npy", allow_pickle=True)
        self.auc = np.load(f"/home/smoothjazzuser/Desktop/videogame-anomoly/MNAD/exp/{self.dataset}/{self.method}/log/auc.npy", allow_pickle=True)
        self.feature_distance_list = np.load(f"/home/smoothjazzuser/Desktop/videogame-anomoly/MNAD/exp/{self.dataset}/{self.method}/log/feature_distance_list.npy", allow_pickle=True)
        self.psnr_list = np.load(f"/home/smoothjazzuser/Desktop/videogame-anomoly/MNAD/exp/{self.dataset}/{self.method}/log/psnr_list.npy", allow_pickle=True)
        self.label_list = np.load(f"/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/frame_labels_{self.dataset}.npy", allow_pickle=True).tolist()[0]
        self.x = self.anomaly_score_total_list
        self.name = self.key = list(np.array(self.psnr_list).reshape(1).tolist()[0].keys())[0]
        print('len x',len(self.x),"len y",len(self.label_list), 'auc:',self.auc, "key:", self.key)
        ############################################################################################################
        self.psnr = normalize_list(np.array(self.psnr_list).reshape(1).tolist()[0][self.name])
        self.feature_distance = normalize_list(self.feature_distance_list.tolist()[self.name])
        self.adjustment = 4 if self.method == 'pred' else 0
        self.x = self.x[self.adjustment:]
        self.label_list = self.label_list[self.adjustment:]
        self.psnr = self.psnr[self.adjustment:]
        self.feature_distance = self.feature_distance[self.adjustment:]
        self.x_shape = self.x.shape[0]
        self.start = 0
        self.end = int(self.x_shape)
        ############################################################################################################
        if self.metric == 'anomaly_score':
            self.x = self.x[self.start:self.end]
        elif self.metric == 'feature_distance':
            self.x = self.feature_distance[self.start:self.end]
        elif self.metric == 'psnr':
            self.x = self.psnr[self.start:self.end]

        self.label_list = self.label_list[self.start:self.end]
        ############################################################################################################
        self.label_points = {'x':[], 'y':[]}
        self.label_plot = {'x':[], 'y':[]}
        for i in range(len(self.label_list)):
            if self.label_list[i] == 1:
                self.label_points['x'].append(i)
                self.label_points['y'].append(self.x[i])
                
                self.label_plot['y'].append(self.x[i])
            else:
                self.label_plot['y'].append(0)
            self.label_plot['x'].append(i)
        ############################################################################################################
        self.adwin = ADWIN(.05)
        self.drift = {'x': [], 'y': []}
        self.xpos = 0
        for i in range(self.x_shape):  # number of frames in vide0
            self.g = self.adwin.add_element(self.x[i])
            if self.adwin.detected_change():
                self.drift['y'].append(self.x[i])
                self.drift['x'].append(self.xpos)
                self.adwin.reset()
            elif self.adwin.detected_warning_zone():
                self.drift['y'].append(self.x[i])
                self.drift['x'].append(self.xpos)
            self.xpos += 1
        ############################################################################################################
        # plot the changes
        # self.fig, self.ax = plt.subplots()
        # self.fig.set_size_inches(200.5, 7.5, forward=True)
        # plt.xticks(np.arange(0, self.end+1, 10.0))

        # self.ax.plot(np.arange(len(self.x)), self.x, color='black', label='anomaly score', linewidth=0.6)
        # self.ax.scatter(np.arange(len(self.x)), self.x, color='black', label='anomaly score', s=4)
        # self.ax.scatter(self.label_points['x'], self.label_points['y'], color='red', marker='x', s=10)
        # # ax.scatter(drift['x'], drift['y'],color='green', marker='|', s=1000000)
        # self.ax.legend([self.metric,self.metric,'ground_truth'])
        # plt.savefig(f"/home/smoothjazzuser/Desktop/{self.dataset}_{self.method}_{self.metric}_folder-{self.name}_std_correction-{self.std_correction}.svg")
        # plt.show()
        ############################################################################################################
        stock, start, end, x, y = False, False, False, self.x, self.label_points
        self.data = x
        self.data = self.normalize_data(self.data)
        self.label_points = y
        self.data = pd.Series(self.data)
        self.label_points = pd.DataFrame.from_dict(self.label_points, orient='index')
        self.label_points = self.label_points.transpose() 
        self.zeros = pd.DataFrame(np.zeros(self.data.shape[0]), columns=['zeros'])
    
        ############################################################################################################
        
        self.plot(window=10, alpha=0.5,  plots = ['bet_std_mean', 'std', 'data','mean'])
        self.show_plot()
        self.plot(window=10, alpha=0.5,  plots = ['bet_std_mean', 'std','mean'])
        self.show_plot()

        self.r = int((len(self.label_list)-1)/1)
        self.preds = sorted(glob(f"/home/smoothjazzuser/Desktop/videogame-anomoly/MNAD/exp/{self.dataset}/{self.method}/log/preds/*.jpg"), key=lambda x: int(x.split('/')[-1].split('.')[0]))
        self.diffs = sorted(glob(f"/home/smoothjazzuser/Desktop/videogame-anomoly/MNAD/exp/{self.dataset}/{self.method}/log/diffs/*.jpg"), key=lambda x: int(x.split('/')[-1].split('.')[0]))
        self.ground_truth = []
        self.folders = sorted(glob(f"/home/smoothjazzuser/Desktop/videogame-anomoly/MNAD/dataset/{self.dataset}/testing/frames/*/"), key=lambda x: int(x.split('/')[-2]))
        self.files = [sorted(glob(f"{file}*.jpg"), key=lambda x: int(x.split('/')[-1].split('.')[0])) for file in self.folders]
        for folder in sorted(glob(f"/home/smoothjazzuser/Desktop/videogame-anomoly/MNAD/dataset/{self.dataset}/testing/frames/*/"), key=lambda x: int(x.split('/')[-2])):
            for file in sorted(glob(f"{folder}*.jpg"), key=lambda x: int(x.split('/')[-1].split('.')[0])):
                self.ground_truth.append(plt.imread(file))

        self.preds = [plt.imread(pred) for pred in self.preds]
        self.diffs = [plt.imread(diff) for diff in self.diffs]
        !rm -rf /tmp/*.jpg
        !rm -rf /tmp/*.jpg
        !rm -rf /tmp/*.svg
        #split r into 10 chunks
        self.rr = list(range(self.r))
        a = Parallel(n_jobs=12, backend='multiprocessing')(delayed(plot_images)(i, self.x, self.diffs[i], self.ground_truth[i], self.label_plot, self.preds[i], self.points) for i in self.rr[0:int(len(self.rr)/2)])
        del a
        gc.collect()
        a = Parallel(n_jobs=12, backend='multiprocessing')(delayed(plot_images)(i, self.x, self.diffs[i], self.ground_truth[i], self.label_plot, self.preds[i], self.points) for i in self.rr[int(len(self.rr)/2):-1])
        del a
        gc.collect()
        del self.preds
        del self.diffs
        del self.ground_truth
        gc.collect()
        self.a = imageio.imread(f"/tmp/0.jpg").shape
        self.h,self.w = self.a[0], self.a[1]
        self.frames = Parallel(n_jobs=12, backend='multiprocessing')(delayed(load_frames)(i, self.h, self.w) for i in tqdm(range(self.r-1))) 
        !rm -rf /tmp/*.jpg
        !rm -rf /tmp/*.jpg
        !rm -rf /tmp/*.svg
        #delete all variables except frames
        gc.collect()
        imageio.mimsave(f'/home/smoothjazzuser/Desktop/{self.dataset}_{self.method}_{self.metric}_folder-{self.name}_std_correction-{self.std_correction}.mp4', self.frames, fps=10)
        del self.frames
        gc.collect()
    def normalize_data(self, data):
        #return data
        return (data - data.min()) / (data.max() - data.min())
    def download_data(self, ticker, start_date, end_date):
        self.data = pdr.get_data_yahoo(ticker, start_date, end_date)
        self.data = self.data.dropna()
        print(self.data.head())
        #normalize between 0 and 1
        self.data = self.normalize_data(self.data['Adj Close'])
        return self.data
    def rolling_average(self, data, window):
        roll = data.rolling(window).mean()
        #normalize between 0 and 1
        roll = self.normalize_data(roll)
        # correct for window size shift
        #roll = roll.shift(-window)
        return  roll
    def rolling_std(self, data, window):
        roll = data.rolling(window).std()
        #normalize between 0 and 1
        roll = self.normalize_data(roll)
        # correct for window size shift
        #roll = roll.shift(-window)
        return roll
    def gradient_at_each_point(self, data, window=False):
        if window: 
            self.gradient = np.gradient(data.rolling(window))
        else:
            self.gradient = np.gradient(self.data)
        self.gradient = pd.Series(self.gradient)
        #return gradient
    def points_above_rollingaverage_and_bellow_rollingstd(self, data, window, rolling=False):
        if rolling:
            rolling_mean = self.rolling_average(data, window)
            self.gradient_at_each_point(rolling_mean, window)
            rolling_std = self.rolling_std(data, window)
            points = data[(data > rolling_mean) & (data < rolling_std) & (self.gradient <= self.zeros.zeros)]
        else:
            rolling_std = self.rolling_std(data, window)
            self.gradient_at_each_point(data, False)
            points = data[(data < rolling_std) & (self.gradient <= self.zeros.zeros)]
        return points
    def plot(self, window=20, alpha=0.5, plots = ['data', 'mean', 'std', 'exponential', 'troughs', 'bet_std_mean', 'gradient']):
        plt.figure(figsize=(int(len(self.data)/76*1.5), 7))

        if 'mean' in plots:
            rolling_mean = self.rolling_average(self.data, window)
            plt.plot(rolling_mean, label='Rolling Mean')
        if 'std' in plots:
            rolling_std = self.rolling_std(self.data, window)
            plt.plot(rolling_std, label='Rolling Std', color='green')
        if 'exponential' in plots:
            exponential_smoothing = self.exponential_smoothing(self.data, alpha)
            plt.plot(exponential_smoothing, label='Exponential Smoothing')
        if 'troughs' in plots:
            troughs = self.lines_at_troughs(self.data, window)
            plt.plot(troughs, label='Lines at Troughs')
        if 'bet_std_mean' in plots:
            self.points = self.points_above_rollingaverage_and_bellow_rollingstd(self.data, window, rolling=False)
            # point y values should equal to data y values
            plt.plot(self.points, 'ro', label='Flagged points', color='purple', markersize=10, alpha=0.5, marker='o')
            # plot these points with y-values = data y values
        if 'data' in plots:
            plt.plot(np.arange(len(self.data)), self.data, color='black', label='data', linewidth=0.6)
            plt.scatter(np.arange(len(self.data)), self.data, color='black', label='data', s=4)
            plt.scatter(self.label_points['x'], self.label_points['y'], color='red', marker='x', s=10)
        plt.legend(loc='upper left')
        #return all plots
        self.all_plots = plt.gcf()
        #del plot
        plt.close('all')
        return self.all_plots
    def show_plot(self):
        plt.show()
        return self.all_plots
    ###############################################################################

        
    
run = anomaly(method='recon', metric='anomaly_score', std_correction='false', dataset="bugs")

In [ ]:
run.swap_test_folders('01')

In [ ]:
#run.cut_dataset(keep_every=15)

In [ ]:
run.Evaluate()

In [ ]:
# analysis
#run.analysis()

In [ ]:
#run.rm_train_music_copy()